In [1]:
from xcube_smos.nckcindex.nckcindex import NcKcIndex
import json

This notebook demonstrates the Python API of the **SMOS Level-2 Kerchunk Index**.

Note, there is also a CLI version of this API, refer to:

In [2]:
!nckcidx --help

Usage: nckcidx [OPTIONS] COMMAND [ARGS]...

  Manage NetCDF Kerchunk indexes.

  A NetCDF Kerchunk index is a directory that contains references to NetCDF
  files stored in S3. For every NetCDF file in S3, a Kerchunk JSON file is
  created and saved. The JSON files are stored in a directory tree that
  corresponds to the NetCDF file S3 key prefix relative to the bucket name.

  This form of the NetCDF Kerchunk index is useful for NetCDF files that
  represent datasets that cannot easily be concatenated along a given or new
  dimension, e.g., SMOS Level-2 products.

Options:
  --debug      Output verbose debugging information. NOT IMPLEMENTED YET.
  --traceback  Output stack traceback on errors. NOT IMPLEMENTED YET.
  --help       Show this message and exit.

Commands:
  create  Create a NetCDF Kerchunk index.
  info    Inform about a NetCDF Kerchunk index.
  sync    Synchronize a NetCDF Kerchunk index.


In [3]:
with open("creodias-credentials.json") as f:
    credentials = json.load(f)

# Options for the NetCDF source files on CreoDIAS    
s3_options = dict(
    **credentials,
    endpoint_url="https://s3.cloudferro.com", 
)  

Create a local test index in default folder `./nckc-index`. After execution, find index folder in file browser.  

In [4]:
fi = NcKcIndex.create(s3_options=s3_options,
                      replace_existing=True)

Show local index path.

In [5]:
fi.index_path

'nckc-index'

Show index configuration. Commented out here because it (still) contains credentials.

In [6]:
#fi.index_config

Open the local test index from default folder `./nckc-index`. It is still empty.    

In [7]:
fi = NcKcIndex.open()
fi.index_path

'nckc-index'

Now we are going to index some NetCDF files. We first perform a dry run to check what will happen:

In [8]:
%%timeit -n 1 -r 1
fi.sync(prefix='SMOS/L2SM/MIR_SMUDP2/2023/03/01/', dry_run=True)

Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T033242_20230301T041728_700_001_1/SM_OPER_MIR_SMUDP2_20230301T033242_20230301T041728_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T053947_20230301T060606_700_001_1/SM_OPER_MIR_SMUDP2_20230301T053947_20230301T060606_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T074300_20230301T083612_700_001_1/SM_OPER_MIR_SMUDP2_20230301T074300_20230301T083612_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T092303_20230301T101617_700_001_1/SM_OPER_MIR_SMUDP2_20230301T092303_20230301T101617_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T110308_20230301T115622_700_001_1/SM_OPER_MIR_SMUDP2_20230301T110308_20230301T115622_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T133310_20230301T142630_700_00

Looks good so we are now going to write index files to disk:

In [9]:
%%timeit -n 1 -r 1
fi.sync(prefix='SMOS/L2SM/MIR_SMUDP2/2023/03/01/')

Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T033242_20230301T041728_700_001_1/SM_OPER_MIR_SMUDP2_20230301T033242_20230301T041728_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T053947_20230301T060606_700_001_1/SM_OPER_MIR_SMUDP2_20230301T053947_20230301T060606_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T074300_20230301T083612_700_001_1/SM_OPER_MIR_SMUDP2_20230301T074300_20230301T083612_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T092303_20230301T101617_700_001_1/SM_OPER_MIR_SMUDP2_20230301T092303_20230301T101617_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T110308_20230301T115622_700_001_1/SM_OPER_MIR_SMUDP2_20230301T110308_20230301T115622_700_001_1.nc
Indexing s3://EODATA/SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T133310_20230301T142630_700_00

Submitting the same prefix once more will omit writing any new indexes:

In [10]:
%%timeit -n 1 -r 1
fi.sync(prefix='SMOS/L2SM/MIR_SMUDP2/2023/03/01/')

Skipping SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T033242_20230301T041728_700_001_1/SM_OPER_MIR_SMUDP2_20230301T033242_20230301T041728_700_001_1.nc, index exists
Skipping SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T053947_20230301T060606_700_001_1/SM_OPER_MIR_SMUDP2_20230301T053947_20230301T060606_700_001_1.nc, index exists
Skipping SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T074300_20230301T083612_700_001_1/SM_OPER_MIR_SMUDP2_20230301T074300_20230301T083612_700_001_1.nc, index exists
Skipping SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T092303_20230301T101617_700_001_1/SM_OPER_MIR_SMUDP2_20230301T092303_20230301T101617_700_001_1.nc, index exists
Skipping SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T110308_20230301T115622_700_001_1/SM_OPER_MIR_SMUDP2_20230301T110308_20230301T115622_700_001_1.nc, index exists
Skipping SMOS/L2SM/MIR_SMUDP2/2023/03/01/SM_OPER_MIR_SMUDP2_20230301T133310_20230301T142630_700_001_